In [ ]:
# Copyright (c) 2022, salesforce.com, inc.
# All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause

This notebook can be run directly on Google Colab. 

Notice that you need to specify the directory/filename to save the document projection vectors.

To read or write files on Google Drive, you can run the code below to mount your Google Drive in the notebook:

```
from google.colab import drive
drive.mount('/content/drive')
```
Then you can specify the path to the file on the Google drive by a path string starting with "/content/drive/MyDrive/"

In [ ]:
# !pip install datasets

In [ ]:
'''
  Load data.
  You can either download the data again, or just reload the data which has been
  saved as doc.jsonl from 02-doc_process (train+test)
'''
doc_list = []

'''option 1: download the mnli and reconstruct the input'''
# from datasets import load_dataset
# dataset = load_dataset('multi_nli', split='validation_matched')
# dataset = dataset.filter(lambda x: x['genre']=='travel')
# for item in dataset:
#     doc_list.append(item['premise']+'<S/>'+item['hypothesis'])

'''option 2: read the json file and reconstruct the input'''
import pandas as pd
df = pd.read_json(path_or_buf="<specify your path>/mnli_government_travel/doc.jsonl", orient="records")
for idx, item in df.iterrows():
    doc_list.append(item['sentence1']+'<S/>'+item['sentence2'])

In [ ]:
doc_list[:2]

["Calcutta seems to be the only other production center having any pretensions to artistic creativity at all, but ironically you're actually more likely to see the works of Satyajit Ray or Mrinal Sen shown in Europe or North America than in India itself.<S/>Most of Mrinal Sen's work can be found in European collections.",
 'The most important directions are simply up and up leads eventually to the cathedral and fortress commanding the hilltop, and down inevitably leads to one of three gates through the wall to the new town.<S/>Go downwards to one of the gates, all of which will lead you into the cathedral.']

Generate Document Embedding

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 5.4 MB/s 
     |████████████████████████████████| 3.8 MB 65.3 MB/s 
     |████████████████████████████████| 1.2 MB 45.1 MB/s 
     |████████████████████████████████| 6.5 MB 48.0 MB/s 
     |████████████████████████████████| 895 kB 54.1 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=5f6e6fdabdbb1d90aeaff06cf53c8d480e37d1a8540aa5db89c2823e79a0fbbd
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [ ]:
sentence_embeddings = model.encode(doc_list)

In [ ]:
sentence_embeddings.shape

(1976, 768)

## Dimensionality Reduction 
run t-SNE algorithm to map the document embedding to 2D space.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X_embedded = TSNE(n_components=2).fit_transform(sentence_embeddings)

In [ ]:
np.savetxt("<spcify your path>/mnli_government_travel/sentence_tsne.csv", X_embedded, delimiter=",")